### Task 2 of 4

List of API queries, which will be conducted:

1) Query of a Point

2) Query of an Area

3) Planet Data Query

Research: https://www.programmableweb.com/category/marine/api  <-- 54 marine APIs

### API 1: Stormglass API

* Via this API you can fetch weather data for any coordinate or area on the globe (but also Tidal, Astronomic and Historic Data )
* The approach is a simple and programmatic one.
* Conventional HTTP requests are used 
* When a request is successful, a response will be sent back in the form of a JSON object.

In [2]:
import os, sys
import arrow
import requests
import json
import pandas as pd
import geopandas as gpd
import csv
import curl
from osgeo import ogr
import fiona
import geojson
import urllib
import gdal
import subprocess
import shapefile as shp
from scipy.interpolate import griddata
import planet
from planet import api
import retrying
from requests.auth import HTTPBasicAuth
import jq

In [3]:
os.chdir('/home/nina/Dokumente/_Advanced_Methods/Advanced_MET/PerformanceRecord/Task_2')
os.getcwd()

'/home/nina/Dokumente/_Advanced_Methods/Advanced_MET/PerformanceRecord/Task_2'

In [4]:
# 1 - Point request

# Get first hour of today
start = arrow.now().floor('day')

# Get last hour of today
end = arrow.now().ceil('day')

response = requests.get(
  'https://api.stormglass.io/v1/weather/point',
  params={
    'lat': 33.82,
    'lng': 123.22,
    'params': ','.join(['waveHeight', 'airTemperature']),
    'start': start.to('UTC').timestamp,  # Convert to UTC timestamp
    'end': end.to('UTC').timestamp  # Convert to UTC timestamp
  },
  headers={
    'Authorization': 'db15d3e0-9cc3-11e9-af8c-0242ac130004-db15d4da-9cc3-11e9-af8c-0242ac130004'
  }
)

# Do something with response data.
json_data = response.json()

In [8]:
json_data # hourly data of that one point, in case you want to go surfing there ;-)

{'hours': [{'airTemperature': [{'source': 'sg', 'value': 26.3},
    {'source': 'noaa', 'value': 26.3}],
   'time': '2019-08-28T22:00:00+00:00',
   'waveHeight': [{'source': 'sg', 'value': 0.77},
    {'source': 'meteo', 'value': 0.77},
    {'source': 'icon', 'value': 0.89},
    {'source': 'noaa', 'value': 0.81}]},
  {'airTemperature': [{'source': 'sg', 'value': 26.4},
    {'source': 'noaa', 'value': 26.4}],
   'time': '2019-08-28T23:00:00+00:00',
   'waveHeight': [{'source': 'sg', 'value': 0.77},
    {'source': 'meteo', 'value': 0.77},
    {'source': 'icon', 'value': 0.89},
    {'source': 'noaa', 'value': 0.82}]},
  {'airTemperature': [{'source': 'sg', 'value': 26.51},
    {'source': 'noaa', 'value': 26.51}],
   'time': '2019-08-29T00:00:00+00:00',
   'waveHeight': [{'source': 'sg', 'value': 0.77},
    {'source': 'meteo', 'value': 0.77},
    {'source': 'icon', 'value': 0.89},
    {'source': 'noaa', 'value': 0.82}]},
  {'airTemperature': [{'source': 'sg', 'value': 26.35},
    {'source': 

In [9]:
with open('./stormglass_point.geojson', 'w') as outfile:  
    geojson.dump(json_data, outfile)

In [10]:
# Transform JSON point into csv (which is then uploadable into QGIS)
# formating via loops

d = open('./stormglass_point.geojson')
data = json.load(d)
d.close()

#print(data)

with open('./stormglass_point.csv', 'w') as csvfile:
    fieldnames = ['lat', 'long','time','AirTemperature_SG', 'AirTemperature_NOAA', 'WaveHeight_SG', 'WaveHeight_Meteo', 'WaveHeight_Icon', 'WaveHeight_NOAA']
    w = csv.DictWriter(csvfile,delimiter=',', fieldnames=fieldnames)
    w.writeheader()

    for rec in data['hours']:
        if 'time' in rec: u = str(rec['time'])

        t1src = rec['airTemperature'][0]['source']
        t1 = rec['airTemperature'][0]['value']
        t2src = rec['airTemperature'][1]['source']
        t2 = rec['airTemperature'][1]['value']
        #print(t1src, t1, t2src, t2)
        
        w1src = rec['waveHeight'][0]['source']
        w1 = rec['waveHeight'][0]['value']
        w2src = rec['waveHeight'][1]['source']
        w2 = rec['waveHeight'][1]['value']
        w3src = rec['waveHeight'][2]['source']
        w3 = rec['waveHeight'][2]['value']
        w4src = rec['waveHeight'][3]['source']
        w4 = rec['waveHeight'][3]['value']
        #print(w1src, w1, w2src, w2, w3src, w3, w4src, w4)

        
        w.writerow({'lat' : 33.82, 'long': 123.22, 'time': u, 'AirTemperature_SG': t1 , 'AirTemperature_NOAA': t2, 'WaveHeight_SG': w1, 'WaveHeight_Meteo': w2, 'WaveHeight_Icon': w3, 'WaveHeight_NOAA': w4})


#################################################################################################################################
#################################################################################################################################
#################################################################################################################################

In [5]:
# 2 - Area request
response = requests.get(
  'https://api.stormglass.io/v1/weather/area',
  params={
      'box': '36,121:29,127', # 121 , 36: 127, 29
      'params': 'currentDirection,currentSpeed,waterTemperature,windWaveDirection'
  },
  headers={
    'Authorization': 'db15d3e0-9cc3-11e9-af8c-0242ac130004-db15d4da-9cc3-11e9-af8c-0242ac130004'
  }
)

# Do something with response data.
json_data_OceanDirections = response.json()

In [12]:
json_data_OceanDirections

{'grid': {'29.0': {'121.0': [{'time': '2019-08-29T00:00:00+00:00',
     'waterTemperature': 30.11,
     'windWaveDirection': 180.0},
    {'time': '2019-08-29T03:00:00+00:00',
     'waterTemperature': 35.4,
     'windWaveDirection': 20.67},
    {'time': '2019-08-29T06:00:00+00:00',
     'waterTemperature': 23.15,
     'windWaveDirection': 30.74},
    {'time': '2019-08-29T09:00:00+00:00',
     'waterTemperature': 21.79,
     'windWaveDirection': 24.21},
    {'time': '2019-08-29T12:00:00+00:00',
     'waterTemperature': 21.28,
     'windWaveDirection': 26.6},
    {'time': '2019-08-29T15:00:00+00:00',
     'waterTemperature': 20.88,
     'windWaveDirection': 19.92},
    {'time': '2019-08-29T18:00:00+00:00',
     'waterTemperature': 19.45,
     'windWaveDirection': 352.9},
    {'time': '2019-08-29T21:00:00+00:00',
     'waterTemperature': 18.75,
     'windWaveDirection': 6.46}],
   '122.0': [{'currentDirection': 24.54,
     'currentSpeed': 0.33,
     'time': '2019-08-29T00:00:00+00:00',
   

In [13]:
with open('./stormglass.json', 'w') as outfile:  # or json
    json.dump(json_data_OceanDirections, outfile)

In [14]:
# Transform JSON into csv (because it is a simple format :D )
# formating via loops

d = open('./stormglass.json')
data = json.load(d)
d.close()

with open('./stormglass.csv', 'w') as csvfile:
    fieldnames = ['lat', 'long', 'time', 'WaterTemperature', 'WindWaveDirection', 'currentDirection', 'CurrentSpeed']
    w = csv.DictWriter(csvfile,delimiter=',', fieldnames=fieldnames)
    w.writeheader()
    
    for k, v in data['grid'].items():
        #print(k) # k is lat, v is a lot!
        for k2,v2 in v.items():
            #print(" "+k2) # k2 are long
            for o in v2:
                #print(o)
                u = ''
                x = ''
                y = ''
                z = ''
                v = str(o['time'])
                if 'currentSpeed' in o: u = str(o['windWaveDirection'])
                if 'currentDirection' in o: x = str(o['currentDirection'])
                if 'waterTemperature' in o: y = str(o['waterTemperature'])
                if 'windWaveDirection' in o: z = str(o['windWaveDirection'])

                w.writerow({'lat' : k, 'long': k2, 'time':  v,'WaterTemperature': y, 'WindWaveDirection': z,'currentDirection': x,'CurrentSpeed': u})


#################################################################################################################################
#################################################################################################################################
#################################################################################################################################

### API 2: PLANET Data API
https://developers.planet.com/docs/quickstart/searching-for-imagery/

**1 Install required packages**

* pip install retrying, requests, planet, jq
* npm install -g geojsonio-cli

**2 Export planet API key to variable**

In [21]:
!export PL_API_KEY=#######################

**3 Searching for imagery** - focus on metadata

In [117]:
!curl -L -H PL_API_KEY 'https://api.planet.com/data/v1/item-types' | jq '.item_types[]?.id'
"REOrthoTile"
"PSOrthoTile"

# Error solved: https://stackoverflow.com/questions/28213232/docker-error-jq-error-cannot-iterate-over-null

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    78  100    78    0     0    139      0 --:--:-- --:--:-- --:--:--   139


'PSOrthoTile'

In [118]:
geo_json_geometry = {
  "type": "Polygon",
  "coordinates": [
    [
      [
        126.01,        # spatial extent I desire
        33.10
      ],
      [
        126.10,
        33.42
      ],
      [
        126.41,
        33.22
      ],
      [
        126.23,
        33.07      
      ],
      [
        126.15,
        33.05
      ]
    ]
  ]
}

# filter for items the overlap with our chosen geometry
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geo_json_geometry
}

# filter images acquired in a certain date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": "2019-07-01T12:00:00.000Z",          # desired temporal extent I like
    "lte": "2019-08-15T00:00:00.000Z"
  }
}

# filter any images which have less than 0.25% clouds
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.25
  }
}

# create a filter that combines the geo and date filters
# could also use an "OrFilter"
water_area = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

In [129]:
stats_endpoint_request = {"interval": "day", "item_types": ["PSOrthoTile"], "filter": water_area}
result = requests.post('https://api.planet.com/data/v1/stats', auth=HTTPBasicAuth(PL_API_KEY, ''), 
                       json=stats_endpoint_request)

print(result.text)

{"utc_offset": "+0h", "interval": "day", "buckets": [{"count": 11, "start_time": "2019-07-04T00:00:00.000000Z"}, {"count": 0, "start_time": "2019-07-05T00:00:00.000000Z"}, {"count": 9, "start_time": "2019-07-06T00:00:00.000000Z"}, {"count": 0, "start_time": "2019-07-07T00:00:00.000000Z"}, {"count": 0, "start_time": "2019-07-08T00:00:00.000000Z"}, {"count": 0, "start_time": "2019-07-09T00:00:00.000000Z"}, {"count": 0, "start_time": "2019-07-10T00:00:00.000000Z"}, {"count": 0, "start_time": "2019-07-11T00:00:00.000000Z"}, {"count": 1, "start_time": "2019-07-12T00:00:00.000000Z"}, {"count": 0, "start_time": "2019-07-13T00:00:00.000000Z"}, {"count": 3, "start_time": "2019-07-14T00:00:00.000000Z"}, {"count": 8, "start_time": "2019-07-15T00:00:00.000000Z"}, {"count": 11, "start_time": "2019-07-16T00:00:00.000000Z"}, {"count": 0, "start_time": "2019-07-17T00:00:00.000000Z"}, {"count": 0, "start_time": "2019-07-18T00:00:00.000000Z"}, {"count": 0, "start_time": "2019-07-19T00:00:00.000000Z"}, {

In [130]:
search_endpoint_request = {"item_types": ["PSOrthoTile"],"filter": water_area}

result = requests.post('https://api.planet.com/data/v1/quick-search', auth=HTTPBasicAuth(PL_API_KEY, ''), 
                       json=search_endpoint_request)

print(result.text)

{"_links": {"_first": "https://api.planet.com/data/v1/searches/b56938489fb44d949f0b553477352246/results?_page=eyJxdWVyeV9wYXJhbXMiOiB7fSwgInNvcnRfcHJldiI6IGZhbHNlLCAicGFnZV9zaXplIjogMjUwLCAic29ydF9ieSI6ICJwdWJsaXNoZWQiLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9kZXNjIjogdHJ1ZX0%3D", "_next": "https://api.planet.com/data/v1/searches/b56938489fb44d949f0b553477352246/results?_page=eyJxdWVyeV9wYXJhbXMiOiB7fSwgInNvcnRfcHJldiI6IGZhbHNlLCAicGFnZV9zaXplIjogMjUwLCAic29ydF9ieSI6ICJwdWJsaXNoZWQiLCAic29ydF9zdGFydCI6ICIyMDE5LTA3LTA0VDA2OjMyOjUwLjM5NDAwMFoiLCAic29ydF9sYXN0X2lkIjogIjI0OTU2NDJfNTI1NDQwNF8yMDE5LTA3LTA0XzEwMTAiLCAic29ydF9kZXNjIjogdHJ1ZX0%3D", "_self": "https://api.planet.com/data/v1/searches/b56938489fb44d949f0b553477352246/results?_page=eyJxdWVyeV9wYXJhbXMiOiB7fSwgInNvcnRfcHJldiI6IGZhbHNlLCAicGFnZV9zaXplIjogMjUwLCAic29ydF9ieSI6ICJwdWJsaXNoZWQiLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9kZXNjIjogdHJ1ZX0%3D"}, "features": [{"_links": {"_self"

Conclusion: **Searching for Imagery was successfull!**

**4 Downloading imagery**

e.g.: ItemType: PSOrthoTile

ItemId: 2581465_5254404_2019-08-07_1058

Link: https://developers.planet.com/docs/quickstart/downloading-imagery/

In [54]:
## Accessing a Single Item
# Visualize the item before you downlad it by extracting the geometry coordiates (footprint)
!curl -L -H "Authorization: api-key $PL_API_KEY" \
    'https://api.planet.com/data/v1/item-types/PSOrthoTile/items/2596777_5154327_2019-08-13_0f15' \
    | jq '.geometry' | geojsonio

# Possible Improvement: Solve Error

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    78  100    78    0     0     85      0 --:--:-- --:--:-- --:--:--    85
This is not valid GeoJSON. Errors:

The root of a GeoJSON object must be an object.


In [55]:
## Asset Types
# A single satellite image can provide many different formats. 
# The existing formats can be demanded and displayed using:
!curl -L -H PL_API_KEY 'https://api.planet.com/data/v1/item-types/PSOrthoTile/items/2596777_5154327_2019-08-13_0f15/assets' | jq 'keys'

# should be: ["analytic", "analytic_xml", "udm", "visual", "visual_xml"]

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    78  100    78    0     0    130      0 --:--:-- --:--:-- --:--:--   130
[
  "errors",
  "message"
]


In [56]:
## Asset Activation
!curl -L -H "Authorization: api-key $PL_API_KEY" \
    'https://api.planet.com/data/v1/item-types/PSOrthoTile/items/2596777_5154327_2019-08-13_0f15/assets/' \
    | jq .visual.status

# should be: "inactive"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    78  100    78    0     0    150      0 --:--:-- --:--:-- --:--:--   150
null


In [62]:
# Activation takes time - in particular if the amount of assets to activate is high
item_id = "2596777_5154327_2019-08-13_0f15"
item_type = "PSOrthoTile"
asset_type = "visual"

# setup auth
session = requests.Session()
session.auth = ("Authorization: api-key $PL_API_KEY", '')

# request an item
item = \
  session.get(
    ("https://api.planet.com/data/v1/item-types/" +
    "{}/items/{}/assets/").format(item_type, item_id))

# extract the activation url from the item for the desired asset
item_activation_url = item.json()[asset_type]["_links"]["activate"]

# request activation
response = session.post(item_activation_url)

print(response.status_code)

KeyError: 'visual'

In [63]:
## Asset Downloading
!curl -L -H "Authorization: api-key $PL_API_KEY" \
    'https://api.planet.com/data/v1/item-types/PSOrthoTile/items/2596777_5154327_2019-08-13_0f15' \
    | jq .visual.status
# should be: "active"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    78  100    78    0     0     75      0  0:00:01  0:00:01 --:--:--    75
null


In [64]:
!curl -L -H "Authorization: api-key $PL_API_KEY" \
    'https://api.planet.com/data/v1/item-types/PSOrthoTile/items/2596777_5154327_2019-08-13_0f15' \
    | jq .visual.location

# should return a download link

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    78  100    78    0     0    134      0 --:--:-- --:--:-- --:--:--   134
null


In [ ]:
!curl -L " Place Holder For The Link from Above " > API_Planet_Image.tiff

### Further Reading: 
Best Practices: Large AOIs

Link: https://developers.planet.com/docs/quickstart/best-practices-large-aois/